# Google Location History - Map Activity using Folium, GPS, GPX 



In [3]:
import gpxpy
import pandas as pd
import numpy as np
import math
import pdb
from folium import plugins
import folium
from IPython.display import display, HTML
import branca
from collections import namedtuple
import xml.etree.ElementTree as ET
import os
import gpxpy
from datetime import datetime
import time
import json
from datetime import datetime as dt
from mpl_toolkits.basemap import Basemap
import imageio

%matplotlib inline


# Load Google Data
Code from [this very helpful blog post](http://geoffboeing.com/2016/06/mapping-google-location-history-python/). 
Uses GPS data from two people living in same household.

In [4]:
# load the google location history data
df_gps = pd.read_json('raw_data/Historial de ubicaciones.json')
df_gps['source'] = 'patrick'
# read in steph data and concat
df_gps_steph = pd.read_json('raw_data/steph_location_history.json')
df_gps_steph['source'] = 'steph'

df_gps = pd.concat([df_gps, df_gps_steph])

print('There are {:,} rows in the location history dataset'.format(len(df_gps)))

There are 1,423,552 rows in the location history dataset


In [5]:
# parse lat, lon, and timestamp from the dict inside the locations column
df_gps['lat'] = df_gps['locations'].map(lambda x: x['latitudeE7'])
df_gps['lon'] = df_gps['locations'].map(lambda x: x['longitudeE7'])
df_gps['timestamp_ms'] = df_gps['locations'].map(lambda x: x['timestampMs'])

# convert lat/lon to decimalized degrees and the timestamp to date-time
df_gps['lat'] = df_gps['lat'] / 10.**7
df_gps['lon'] = df_gps['lon'] / 10.**7
df_gps['timestamp_ms'] = df_gps['timestamp_ms'].astype(float) / 1000
df_gps['datetime'] = df_gps['timestamp_ms'].map(lambda x: dt.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
date_range = '{}-{}'.format(df_gps['datetime'].min()[:4], df_gps['datetime'].max()[:4])

# drop columns we don't need, then show a slice of the dataframe
df_gps = df_gps.drop(labels=['locations', 'timestamp_ms'], axis=1, inplace=False)

df_gps['datetime']  = pd.to_datetime(df_gps['datetime'])
df_gps['week'] = df_gps['datetime'].dt.week

print(df_gps.info())
print(df_gps.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1423552 entries, 0 to 8793
Data columns (total 5 columns):
source      1423552 non-null object
lat         1423552 non-null float64
lon         1423552 non-null float64
datetime    1423552 non-null datetime64[ns]
week        1423552 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 65.2+ MB
None
    source        lat         lon            datetime  week
0  patrick  37.767784 -122.429118 2013-09-13 08:59:31    37
1  patrick  37.767786 -122.429121 2013-09-13 08:59:34    37
2  patrick  37.767787 -122.429124 2013-09-13 08:59:35    37
3  patrick  37.767788 -122.429126 2013-09-13 08:59:36    37
4  patrick  37.767793 -122.429139 2013-09-13 09:00:22    37


# Data Cleaning
* 1) Define Before quarantine and after quarantine date range.
* 2) Filter data.
* 3) Create simple plots on a map showing the difference.

In [6]:
#Define date ranges of 3 distinct phases.
pre_q_date_mask = (df_gps['datetime'] >= '2020-01-08') & (df_gps['datetime'] <= '2020-03-15') 
post_q_date_mask = (df_gps['datetime'] >= '2020-03-16') & (df_gps['datetime'] <= '2020-05-01') & (df_gps['lon']>-.35939) &(df_gps['lat']>39.4640776) &(df_gps['lat']<39.47182)    
exercise_q_date_mask =  (df_gps['datetime'] >= '2020-05-02')

#filter data
pre_q_df = df_gps[pre_q_date_mask]
pre_q_df['phase'] = 'pre_q'
post_q_df = df_gps[post_q_date_mask]
post_q_df['phase'] = 'quarantine'
exercise_q_df = df_gps[exercise_q_date_mask]
exercise_q_df['phase'] = 'exercise'

/Users/patricknoonan/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/patricknoonan/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/patricknoonan/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

# Screenshot Function

In [93]:
def get_png_screenshot_of_html(map_name):
    delay=5
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=map_name)
    browser = webdriver.Firefox(executable_path=r'/Users/patricknoonan/geckodriver')

    #set size and position
#     browser.set_window_position(0, 0)
#     browser.set_window_size(1024, 768)
    
    browser.maximize_window()
    
    browser.get(tmpurl)
    time.sleep(delay)
    png_map_name = map_name + '.png'
    browser.save_screenshot(png_map_name)
    browser.quit()

# Pre Q by week

In [278]:
def make_gif_by_phase(df_name, df_string_name, duration=1, make_gif=True, zoom_start=14, draw_radius_stephs=False, show_home_marker=False, map_title=False):
    week_list = list(df_name.week.unique())
    png_list = []
    for week in week_list:
            #isolate df 
        week_df = df_name[df_name.week==week]
        week_map_name = df_string_name + str(week) + '_.html'
        #make map with week name
        scatter_map(week_df, map_name=week_map_name, zoom_start=zoom_start, draw_radius_stephs=draw_radius_stephs, show_home_marker=show_home_marker, map_title=map_title)
        #do screenshots
        get_png_screenshot_of_html(week_map_name)
        png_map_name = week_map_name + '.png'
        png_list.append(png_map_name)
    #make gif
    if make_gif:
        create_gif(png_list, duration)
    return png_list
    

Create entire GIF

In [250]:
#zoom 14 - should have:
#steph and P data pre-q  - BLUE DOTS
#steph data during q   - RED DOTS
#steph and P, exercise phase  - RED DOTS

start_time = time.time()

pre_q_list = make_gif_by_phase(pre_q_df, df_string_name='pre_q_df' , make_gif=False, zoom_start= 14, draw_radius_stephs=False, show_home_marker=True,  map_title=True)
post_q_list = make_gif_by_phase(post_q_df, df_string_name='post_q_df' , make_gif=False, zoom_start= 14, draw_radius_stephs=False, show_home_marker=True, map_title=True)
exercise_q_df_list = make_gif_by_phase(exercise_q_df, df_string_name='exercise_q_df' , make_gif=False, zoom_start= 14, draw_radius_stephs=True, show_home_marker=True, map_title=True)

joinedlist = pre_q_list + post_q_list + exercise_q_df_list
create_gif(joinedlist, 1.85, '05_13_zoom_14_5_12.gif')

elapsed_time = time.time() - start_time
print('elapsed time is:')
print(elapsed_time/60)

elapsed time is:
5.325122364362081


Create GIF


In [205]:

#filenames=['pre_q_map_valencia_week_10_.html.png', 'pre_q_map_valencia_week_11_.html.png']
def create_gif(filenames, duration, gif_name):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    #output_file = 'Gif-%s.gif' % filename.split("\\")[-1].split(".")[0]
    
    imageio.mimsave(gif_name, images, duration=duration)



# Scatter Maps

In [7]:
#tiles link: https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TilesExample.ipynb


def scatter_map (df, map_name='my_quarantine_map.html', zoom_start=14, draw_radius_stephs=False, show_home_marker=False, map_title=False):
    
    mymap = folium.Map( location=[39.461242, -0.3577096], zoom_start=zoom_start, tiles=None) 
    folium.TileLayer('Stamen Toner', name='Stamen Toner').add_to(mymap)
    
    if df.iloc[0].phase =='quarantine':
        #more accurate GPS data, only use Steph
        df = df[df.source=='steph']
    elif df.iloc[0].phase =='exercise':
        #to de-clutter exercise phase, only Patrick data
        df = df[df.source=='patrick']
        
    for index, row in df.iterrows():
        circle_message = str(row.datetime) + '  lat is: ' + str(row.lat)+ '  lon is: ' + str(row.lon)
        if df.iloc[0].phase =='pre_q':
            folium.vector_layers.Circle(location=[row.lat, row.lon], radius=30, color='blue', fill_color='blue', weight=2, fill_opacity=0.3, tooltip=circle_message).add_to(mymap)
        elif df.iloc[0].phase =='quarantine':
            folium.vector_layers.Circle(location=[row.lat, row.lon], radius=30, color='red', fill_color='red', weight=2, fill_opacity=0.3, tooltip=circle_message).add_to(mymap)
        elif df.iloc[0].phase =='exercise':
            folium.vector_layers.Circle(location=[row.lat, row.lon], radius=30, color='red', fill_color='red', weight=2, fill_opacity=0.3, tooltip=circle_message).add_to(mymap)
            
    if draw_radius_stephs:
        folium.vector_layers.Circle(location=[39.470460, -0.357680], radius=1000, color='red', fill_color='red', weight=2, fill_opacity=0.1, tooltip=circle_message).add_to(mymap)
    
    if show_home_marker:
        folium.Marker([39.470460, -0.357680], popup='Home', icon=folium.Icon(color='black',  opacity=0.2, icon_color='white', icon='home', prefix='fa')).add_to(mymap)
    
    if map_title==True:
        #Logic for color of dots and messaging
        if df.iloc[0].phase =='pre_q':
            weeks_before_lockdown = 12 - df.iloc[0].week
            current_week_in_date_format = df.iloc[0].datetime.strftime("%B %-d, %Y")

            title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Week of {current_week_in_date_format}. Normal Life.</b></h3>
                 <h3 align="center" style="font-size:40px"><b>Weeks before Lockdown: {weeks_before_lockdown}  </b></h3>

                 '''.format(weeks_before_lockdown=weeks_before_lockdown, current_week_in_date_format=current_week_in_date_format)
            mymap.get_root().html.add_child(folium.Element(title_html))
        elif df.iloc[0].phase == 'quarantine':
            weeks_of_lockdown =  df.iloc[0].week - 11
            current_week_in_date_format = df.iloc[0].datetime.strftime("%B %-d, %Y")

            title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Week of {current_week_in_date_format}. Life in Lockdown.  Forbidden to leave house (except for buying food). </b></h3>
                 <h3 align="center" style="font-size:40px"><b>Weeks in Lockdown: {weeks_of_lockdown}  </b></h3>

                 '''.format(weeks_of_lockdown=weeks_of_lockdown, current_week_in_date_format=current_week_in_date_format)
            mymap.get_root().html.add_child(folium.Element(title_html))
        elif df.iloc[0].phase == 'exercise':
            weeks_of_lockdown =  df.iloc[0].week - 11
            current_week_in_date_format = df.iloc[0].datetime.strftime("%B %-d, %Y")

            title_html = '''
                 <h3 align="center" style="font-size:20px"><b>Week of {current_week_in_date_format}. Outside walking allowed! (within 1KM radius). </b></h3>
                 <h3 align="center" style="font-size:40px"><b>Weeks in Lockdown: {weeks_of_lockdown}  </b></h3>

                 '''.format(weeks_of_lockdown=weeks_of_lockdown, current_week_in_date_format=current_week_in_date_format)
            mymap.get_root().html.add_child(folium.Element(title_html))

    
#     plugins.Fullscreen(
#         position='topright',
#         title='Expand me',
#         title_cancel='Exit me',
#         force_separate_button=True
#     ).add_to(mymap)

    #folium.LayerControl(collapsed=True).add_to(mymap)
    mymap.save(map_name)
    mymap

In [8]:
scatter_map(pre_q_df, map_name='for_blog_pre_q_mashup.html', zoom_start=13, show_home_marker=True, map_title=False)


In [9]:
%%HTML
<iframe width="1000" height="500" src='for_blog_pre_q_mashup.html'</iframe>

In [10]:
scatter_map(post_q_df, map_name='for_blog_post_q_df_mashup.html', zoom_start=13, show_home_marker=True, map_title=False)


In [11]:
%%HTML
<iframe width="1000" height="500" src='for_blog_post_q_df_mashup.html'</iframe>

In [12]:
scatter_map(exercise_q_df, map_name='for_blog_post_exercise_q_df_mashup.html', zoom_start=13, show_home_marker=True, map_title=False, draw_radius_stephs=True)


In [13]:
%%HTML
<iframe width="1000" height="500" src='for_blog_post_exercise_q_df_mashup.html'</iframe>

# Bring in GPX garmin data for day by day plotting in radius

Use Garmin Export script for command line tool to auto-import GPX tracks: https://github.com/pe-st/garmin-connect-export
Shell command to update garmin tracks: venv.garminexport/src/garminexport/garminbackup.py --backup-dir='/Users/patricknoonan/Jupyter Notebooks/google_locations/activities' psnoonan@gmail.com

In [16]:
def process_gpx_to_df(file_name):

    gpx = gpxpy.parse(open(file_name))  
    
    #(1)make DataFrame
    track = gpx.tracks[0]
    segment = track.segments[0]
    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points):
        data.append([point.longitude, point.latitude,point.elevation,
                     point.time, segment.get_speed(point_idx)])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    gpx_df = pd.DataFrame(data, columns=columns)
    
    #2(make points tuple for line)
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
    
    return gpx_df, points

def get_activity_name(gpx_file):
    #CHILD OF ROOT, WE NEED TO ACCESS THIS: {http://www.topografix.com/GPX/1/1}trk
    root = ET.parse(gpx_file).getroot()
    for elem in root.iter():
        #print(elem.tag, elem.attrib)
        if elem.tag=='{http://www.topografix.com/GPX/1/1}name':
            return elem.text
        
def get_type_activity(gpx_file):
    #CHILD OF ROOT, WE NEED TO ACCESS THIS: {http://www.topografix.com/GPX/1/1}trk
    root = ET.parse(gpx_file).getroot()
    for elem in root.iter():
        #print(elem.tag, elem.attrib)
        if elem.tag=='{http://www.topografix.com/GPX/1/1}type':
            return elem.text
        
def gpx_files_to_df_basic_stats(folder, date_start):
    gpx_files = os.listdir(folder)
    gpx_list = []
    #ignore empty files
    for file in gpx_files:
        full_path_file = folder+file
        date_str = file[:10]
        if file[0] !='.':
            date_object = datetime.strptime(date_str, '%Y-%m-%d').date()
            date_start = datetime.strptime('2020-05-02', '%Y-%m-%d').date()
            if os.path.getsize(full_path_file) > 0 and full_path_file[-3:]=='gpx' and date_object>= date_start:
                activity_name = get_activity_name(full_path_file)
                activity_type_from_gpx = get_type_activity(full_path_file)
                date = get_date_from_gpx(full_path_file)
                #add to new list
                activity_dict = {'activity_name':activity_name, 'path':full_path_file, 'date':date, 'activity_type_from_gpx':activity_type_from_gpx}
                gpx_list.append(activity_dict)  
    return pd.DataFrame(gpx_list)

def parse_garmin_activity_id_json_summary(file):
    pattern = r'_([0-9]{6,11})_summary\.json'
    #pdb.set_trace()
    match = re.search(pattern, file)
    return match.group(1)

def get_date_from_gpx(full_path_file):
    gpx = gpxpy.parse(open(full_path_file))        
    track = gpx.tracks[0]
    segment = track.segments[0]
    for point_idx, point in enumerate(segment.points):
        return point.time
        
def garmin_map(activity, map_name='my_quarantine_map.html', garmin_folder = 'activities/', date_start='2020-05-02', zoom_start=13, draw_radius_stephs=False, include_geo_json=False, show_home_marker=False):
    garmin_activities = gpx_files_to_df_basic_stats(garmin_folder, date_start=date_start)
    garmin_activities = garmin_activities[garmin_activities.activity_type_from_gpx==activity]
    file_name_list = garmin_activities.path.to_list()
    mymap = folium.Map( location=[ 39.46975, -0.37739 ], zoom_start=zoom_start, tiles=None) 

    folium.TileLayer('Stamen Toner', name='Stamen Toner').add_to(mymap)
    folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
    folium.TileLayer('https://stamen-tiles-{s}.a.ssl.fastly.net/toner-background/{z}/{x}/{y}{r}.png', attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', name='toner-background').add_to(mymap)
    folium.TileLayer('https://stamen-tiles-{s}.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}{r}.png', attr='Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', name='toner-lite').add_to(mymap)

    for file_name in file_name_list:
        gpx_df, points = process_gpx_to_df(file_name)
        folium.PolyLine(points, color='red', weight=3.0, opacity=1).add_to(mymap)
    if draw_radius_stephs:
        folium.vector_layers.Circle(location=[39.470460, -0.357680], radius=1000, color='red', fill_color='red', weight=2, fill_opacity=0.1).add_to(mymap)
        
    if include_geo_json==True:
        espana_municipios_geo = r'municipio_valencia.geojson'
        with open(espana_municipios_geo) as geo_json_file:
            espana_municipios_file = json.load(geo_json_file)
        folium.Choropleth(
        geo_data=espana_municipios_file,
        fill_color='red',
        fill_opacity=0.2,
        line_weight=2,
        overlay = True, #another layer as opposed to it's on tile
        highlight=False, #changes color as you hover over it
        name='Municipio De Valencia',
        show=True,
        ).add_to(mymap)
        
    if show_home_marker: 
        folium.Marker([39.4639845, -0.3517744], popup='Home', icon=folium.Icon(color='black',  opacity=0.2, icon_color='white', icon='home', prefix='fa')).add_to(mymap)

    plugins.Fullscreen(
        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(mymap)

    folium.LayerControl(collapsed=True).add_to(mymap)
    mymap.save(map_name)
    mymap

Auto Import Garmin Activities

In [17]:
garmin_map( activity='hiking', map_name='garmin_hiking_5_13.html', zoom_start=13, draw_radius_stephs=True, include_geo_json=False, show_home_marker=True)

In [18]:
%%HTML
<iframe width="1000" height="500" src='garmin_hiking_5_13.html'</iframe>

In [19]:
garmin_map( activity='cycling', map_name='garmin_cycling_5_13.html', zoom_start=13, draw_radius_stephs=False, include_geo_json=True, show_home_marker=True)


In [20]:
%%HTML
<iframe width="1000" height="500" src='garmin_cycling_5_13.html'</iframe>